In [ ]:
# Define HTTP request, symbols and functions
import json
import pandas as pd
from helper import load_config, fetch_and_save

# Load the configuration
config = load_config()

api_key, symbols = config['api_key'], config['symbols']

functions = ['CASH_FLOW', 'INCOME_STATEMENT', 'BALANCE_SHEET', 'EARNINGS']

In [ ]:
# WARNING: Make actual API calls, may expose to API rate limit
# api_key = os.getenv('API_KEY')

# Loop through and call the function for each symbol and function
for symbol in symbols:
    for function in functions:
        fetch_and_save(symbol, function, api_key)

In [ ]:
# Consolidate into one report
## Dictionary to store the DataFrames
report_data_dict = {}

for symbol in symbols:
     for function in functions:
        
        file_path = f'../../data/fundamental/{symbol}-{function}.json'
        
        # Open and load the JSON file
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        
        # Extract the 'annualReports' data
        report_data = data.get('annualReports', [])
        
        if not report_data:
            report_data = data.get('annualEarnings', [])
        
        # Loop through the report data and reformat it
        for item in report_data:
            fiscal_date = item.get('fiscalDateEnding')
            # Add each key-value pair to the dictionary
            for key, value in item.items():
                if key != 'fiscalDateEnding':
                    if key not in report_data_dict:
                        report_data_dict[key] = {}
                    report_data_dict[key][fiscal_date] = value
        
        # Convert the dictionary to a DataFrame
        df = pd.DataFrame(report_data_dict).T
        
        # original_df = df.copy()
        
        df = df.apply(pd.to_numeric, errors='coerce')
        
        df = df.fillna(0)
        
        df.to_json(f'../../data/consolidated/{symbol}.json', indent=4)